# Base64 algorithm

In [1]:
import numpy as np

In [ ]:
BASE64_TABLE = [
  'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
  'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
  'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
  'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f',
  'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
  'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
  'w', 'x', 'y', 'z', '0', '1', '2', '3',
  '4', '5', '6', '7', '8', '9', '+', '/'
]

In [ ]:
def str_to_bits(string: str) -> np.ndarray:
    bytes_array = bytes(string, 'ascii')
    array = np.frombuffer(bytes_array, dtype=np.uint8)
    bits_array = np.unpackbits(array)
    return bits_array

def bits_to_str(bits: np.ndarray) -> str:
    bits_array = np.packbits(bits)
    return bits_array.tobytes().decode('ascii')

def split_to_sextets(bits: np.ndarray) -> np.ndarray:
    num_blocks = bits.size // 6
    blocks = np.array_split(bits, num_blocks)
    return np.array(blocks, dtype=np.uint8)

def split_to_octets(bits: np.ndarray) -> np.ndarray:
    num_full_subarrays = len(bits) // 8
    subarrays = []
    for i in range(num_full_subarrays):
        subarrays.append(np.array(bits[i * 8: (i + 1) * 8], dtype=np.uint8))
    return subarrays

def add_padding(bits: np.ndarray) -> np.ndarray:
    length = 6 - len(bits) % 6
    return np.append(bits, np.zeros(length))

def bits_to_int(bits_array):
    binary_string = ''.join(bits_array.astype(str))
    return int(binary_string, 2)

def int_to_x_bits(number: int, bits) -> np.ndarray:
    binary_string = np.binary_repr(number, width=bits)
    return np.array(list(binary_string), dtype=np.uint8)

In [ ]:
def encode_base64(string: str) -> str:
    assert len(string) > 0

    pad_len = (3 - (len(string) % 3)) % 3 
    bits = str_to_bits(string)

    if len(string) % 3 > 0:
        bits = add_padding(bits)
    sextets = split_to_sextets(bits)
    result = "".join(BASE64_TABLE[bits_to_int(sextet)] for sextet in sextets)
    result += pad_len * "="
    return result

encode_base64("ab")

In [ ]:
def decode_base64(base64_string: str) -> str:
    assert len(base64_string) > 0

    base64_string = base64_string.replace('=', '')
    bits = np.array([])
    for ch in base64_string:
        index = BASE64_TABLE.index(ch)
        bits = np.append(bits, int_to_x_bits(index, 6))
    
    octets = split_to_octets(bits)
    result = "".join(bits_to_str(octet) for octet in octets)
    return result

decode_base64("YWI=")

In [ ]:
import base64

def assert_base64(testcase):
    assert encode_base64(testcase) == base64.b64encode(testcase.encode('ascii')).decode('ascii')
    assert decode_base64(encode_base64(testcase)) == testcase

assert_base64("a")
assert_base64("ab")
assert_base64("abc")
assert_base64("abcd")
assert_base64("abcde")